In [8]:
%pip install -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [4]:
import folium
import osmnx as ox
import os

In [5]:
# Definicja miejsca: Kabaty, Warszawa, Polska
place_name = "Kabaty, Warsaw, Poland"

# Pobranie grafu ulicznego dla Kabat
G = ox.graph_from_place(place_name, network_type='drive')

# Pobranie krawędzi (ulic) z grafu
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)

# Tworzenie mapy Folium w centrum Kabat
kabaty_coords = [52.135, 21.060]
m = folium.Map(location=kabaty_coords, zoom_start=15)

# Dodanie linii dla każdej ulicy z danych OSM
for _, row in edges.iterrows():
    folium.PolyLine(locations=[(lat, lon) for lon, lat in row['geometry'].coords],
                    color="red", weight=2.5).add_to(m)

# Zapisanie mapy do pliku HTML
os.makedirs("maps", exist_ok=True)
m.save("maps/kabaty_streets.html")

# Wyświetlenie mapy (jeśli w środowisku Jupyter)
m


In [3]:
import os
import numpy as np
import geopandas as gpd
import osmnx as ox
from PIL import Image, ImageDraw

# Definicja miejsca: Kabaty, Warszawa, Polska
place_name = "Kabaty, Warsaw, Poland"

# Pobranie grafu ulicznego dla Kabat
G = ox.graph_from_place(place_name, network_type='drive')

# Pobranie krawędzi (ulic) z grafu
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)

# Ustawienia obrazu
width, height = 500, 500  # Rozmiar obrazu
background_color = (255, 255, 255)  # Kolor tła (biały)
line_color = (255, 0, 0)  # Kolor linii (czerwony)

# Tworzenie nowego obrazu
image = Image.new("RGB", (width, height), background_color)
draw = ImageDraw.Draw(image)

# Ustawienia granic geograficznych dla Kabat
lon_min, lon_max = 21.055, 21.065
lat_min, lat_max = 52.130, 52.140

# Funkcja do konwersji współrzędnych geograficznych na piksele
def geo_to_pixel(lon, lat, width, height, lon_min, lon_max, lat_min, lat_max):
    x = int((lon - lon_min) / (lon_max - lon_min) * width)
    y = height - int((lat - lat_min) / (lat_max - lat_min) * height)  # Inwersja osi y
    return x, y

# Rysowanie ulic na obrazie
for _, row in edges.iterrows():
    coords = row['geometry'].coords
    for i in range(len(coords) - 1):
        x1, y1 = geo_to_pixel(coords[i][0], coords[i][1], width, height, lon_min, lon_max, lat_min, lat_max)
        x2, y2 = geo_to_pixel(coords[i + 1][0], coords[i + 1][1], width, height, lon_min, lon_max, lat_min, lat_max)
        draw.line((x1, y1, x2, y2), fill=line_color, width=1)

# Zapisanie obrazu do pliku PNG
os.makedirs("maps", exist_ok=True)
image.save("maps/kabaty_streets_pixelated.png")
